# Import Required Libraries

In [2]:
import pandas as pd
import os

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
#from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import UnexpectedAlertPresentException, TimeoutException, StaleElementReferenceException, ElementClickInterceptedException, ElementNotInteractableException

import time

from datetime import datetime, date, timedelta

import re

import random

from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow,Flow
from google.auth.transport.requests import Request
import os
import pickle

# Set working directory to be where the chromedriver is

In [3]:
#we need the working directory to be where the chromedriver is

if os.getcwd() != r'C:\Users\path\to\chromedriver':
    os.chdir(r'C:\Users\path\to\chromedriver')

# Main loop that runs the program

In [ ]:
begin_date = date(year = 2020, month = 11, day = 2)
days_to_scrape = 1

for i in range(days_to_scrape):
    scrape_date = begin_date + timedelta(days=i)
    eviction_schedule_df, driver = scrape_clerks_website(scrape_date)
    eviction_schedule_df.to_csv(r'\evictions\eviction schedules\halfway save point\\' + begin_date.strftime('%m-%d-%Y evictions.csv'))
    eviction_schedule_df = contact_info_scrape(eviction_schedule_df, driver)
    folks_to_contact = eviction_schedule_df[eviction_schedule_df['phone_numbers'] != 'N/A']
    folks_to_contact.to_csv(r'\evictions\eviction schedules\\' + begin_date.strftime('%m-%d-%Y eviction schedule.csv'))
    

# In case the program has an issue scraping the contact info, start here

In [ ]:
#start at halfway point
driver = webdriver.Chrome()


#update file name and begin date for whatever date you're working on. 
begin_date = date(year = 2020, month = 11, day = 2)
halfway_save_file = '11-02-2020 evictions.csv'

eviction_schedule_df = pd.read_csv(r'\eviction schedules\halfway save point\\' + halfway_save_file)
eviction_schedule_df = contact_info_scrape(eviction_schedule_df, driver)
folks_to_contact = eviction_schedule_df[eviction_schedule_df['phone_numbers'] != 'N/A']

folks_to_contact.to_csv(r'\evictions\eviction schedules\\' + begin_date.strftime('%m-%d-%Y eviction schedule.csv'))

# Function to scrape the Hamilton County Clerk's website.

**input:** the date of evictions you want to scrape 

**output:** dataframe of relevant data scraped from website, plus the chromedriver.

In [3]:
def scrape_clerks_website(scrape_date):

    eviction_url = 'https://www.courtclerk.org/records-search/eviction-schedule-search/'

    driver = webdriver.Chrome()

    eviction_row_data = []

#    begin_date = date(year = 2020, month = 10, day = 15)

    driver.get(eviction_url)

    scrape_date_str = scrape_date.strftime("%m/%d/%Y")

    proceed = False

    while proceed == False:  
        try:
            date_field = driver.find_element_by_xpath('//*[@id="FFC"]')
            date_field.send_keys(scrape_date_str)

            #fix captcha
            driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/form/input[5]').click()

            WebDriverWait(driver, 4).until(EC.alert_is_present())

            alert = driver.switch_to.alert
            alert.accept()
            driver.refresh()
            time.sleep(5)
        except TimeoutException:
            proceed = True      

    row_num = 1

    while True:

        if len(driver.find_elements_by_xpath('/html/body/div[1]/table[2]/tbody/tr[' + str(row_num) + ']')) == 0:
            break
        else:

            driver.find_element_by_xpath('/html/body/div[1]/table[2]/tbody/tr[' + str(row_num) + ']/td[2]/form/input[2]').click()
            row_num += 4

            time.sleep(3)

            driver.switch_to.window(driver.window_handles[-1])

            #click on party info
            driver.find_element_by_xpath('/html/body/div[1]/table/tbody/tr[1]/td[2]/form[4]/input[3]').click()


            party_info_table = driver.find_element_by_xpath('//*[@id="party_info_table"]')

            for row in party_info_table.find_elements_by_tag_name('tr'):
                if len(row.find_elements_by_tag_name('td')) > 0:
                    if row.find_elements_by_tag_name('td')[2].text == 'D 1':
                        name_str = row.find_elements_by_tag_name('td')[0].text
                        first_name = name_str.split()[0].strip()
                        last_name = name_str.split()[-1].strip()

                        address_str = row.find_elements_by_tag_name('td')[1].text
                        street_addr = address_str.split('\n')[0].strip()
                        city_state_zip = address_str.split('\n')[1]
                        city = city_state_zip.split()[0]
                        state = city_state_zip.split()[1]
                        zip_code = city_state_zip.split()[2]                   

            case_history_icon = driver.find_element_by_xpath('/html/body/div[1]/table/tbody/tr[1]/td[2]/form[1]/input[3]')
            case_history_icon.click()

            time.sleep(3)

            driver.find_element_by_xpath('/html/body/div[1]/table/tbody/tr[2]/td/button').click()

            time.sleep(1)

            case_num = driver.find_element_by_xpath('/html/body/div[1]/table/tbody/tr[1]/td[1]/div[3]/table/tbody/tr[1]/td[2]').text

            amount = driver.find_element_by_xpath('/html/body/div[1]/table/tbody/tr[1]/td[1]/div[3]/table/tbody/tr[7]/td[2]').text

            case_history_table = driver.find_element_by_xpath('//*[@id="case_history_table"]')

            mentions_cdc = False
            requests_interpreter = False

            for row in case_history_table.find_elements_by_tag_name('tr'):
                if len(row.find_elements_by_tag_name('td')) > 1:
                    description_str = row.find_elements_by_tag_name('td')[1].text.lower()
                    if description_str.find('cdc') != -1:
                        mentions_cdc = True

                    if description_str.find('interpreter') != -1:
                        requests_interpreter = True

        eviction_row_data.append([case_num, amount, scrape_date, first_name, last_name, street_addr, city, state, zip_code, requests_interpreter, mentions_cdc])  

        driver.close()
        driver.switch_to.window(driver.window_handles[0])

    col_name_list = ['case_number', 'amount', 'start_date', 'first_name', 'last_name', 'street_address', 'city', 'state', 'zip', 'requested_interpreter', 'mentions_cdc']

    return pd.DataFrame(eviction_row_data, columns = col_name_list), driver
    #driver.close()
    #time.sleep(10)

# Function to scrape contact information of renters
**input:** the dataframe and chromedriver output by scrape_clerks_website

**output:** dataframe with contact information for tenants

In [7]:
def contact_info_scrape(eviction_schedule_df, driver):

    fam_tree_url = 'https://www.familytreenow.com/search'

    df_phone_numbers = []
    df_email_addresses = []

    t_cycle = timedelta(seconds=30)

    for index, row in eviction_schedule_df.iterrows():
        t1 = datetime.now()

        driver.get(fam_tree_url)

        proceed_1 = False
        while proceed_1 == False:

            time.sleep(3)

            add_more_criteria = driver.find_element_by_id('addCriteria')
            try:
                WebDriverWait(driver, 4).until(EC.visibility_of_element_located((By.ID, 'addCriteria')))
                add_more_criteria.click()
                proceed_1 = True
            except ElementClickInterceptedException:
                driver.find_element_by_xpath('/html/body/div[1]/div/a').click()
                add_more_criteria.click()
                proceed_1 = True
            except TimeoutException:
                print("can't click add criteria link.  Refreshing")
                driver.refresh()
                time.sleep(3)

        driver.find_element_by_id('First').send_keys(row['first_name'])
        driver.find_element_by_id('Last').send_keys(row['last_name'])

        city_state_str = row['city'].strip() + ", " + row['state'].strip()

        driver.find_element_by_id('CityStateZip').send_keys(city_state_str)

        WebDriverWait(driver, 4).until(EC.visibility_of_element_located((By.ID, 'StreetAddress')))

        driver.find_element_by_id('StreetAddress').send_keys(row['street_address'])

        #search for person
        if driver.find_elements_by_xpath('/html/body/div[1]/div/a')[0].get_attribute("href") is not None:
            driver.find_element_by_xpath('/html/body/div[1]/div/a').click()

        time.sleep(1)

        try:
            driver.find_element_by_xpath('/html/body/div[3]/div[1]/div/div[1]/div[1]/div/form/div[5]/div/button').click()
        except ElementClickInterceptedException as e:
            driver.find_element_by_class_name("cc-btn.cc-dismiss").click()
#            driver.implicitly_wait(20) 
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[3]/div[1]/div/div[1]/div[1]/div/form/div[5]/div/button')))
            driver.find_element_by_xpath('/html/body/div[3]/div[1]/div/div[1]/div[1]/div/form/div[5]/div/button').click()

        time.sleep(3)

        personal_phone_numbers = []
        personal_email_addresses = []

        #does record exist?
        if len(driver.find_elements_by_xpath('/html/body/div[3]/div[1]/div[2]/div[1]/table/tbody/tr/td/div/div[1]/div[2]/span[2]/a')) == 0:
            found_contact_record = False
            phone_type = 'N/A'
            personal_phone_numbers.append('N/A')
            personal_email_addresses.append('N/A')
        else:
            found_contact_record = True
            driver.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div[1]/table/tbody/tr/td/div/div[1]/div[2]/span[2]/a').click()

            boxes = driver.find_elements_by_class_name('panel-heading.text-center')

            for box in boxes:
                if box.text.lower().find('phone number') != -1:
                    phone_table = box.find_element_by_xpath('following-sibling::table')
                    phone_numbers = phone_table.find_element_by_class_name('table.table-condensed.table-striped')
                    for row in phone_numbers.find_elements_by_tag_name('tr'):
                        if row.find_elements_by_tag_name('td')[1].text.lower().find('wireless') != -1:
                            personal_phone_numbers.append(row.find_elements_by_tag_name('td')[0].text)
                elif box.text.lower().find('email address') != -1:
                    email_table = box.find_element_by_xpath('following-sibling::table')
                    email_addresses = email_table.find_element_by_class_name('table.table-condensed.table-striped')
                    for row in email_addresses.find_elements_by_tag_name('tr'):
                        personal_email_addresses.append(row.find_element_by_tag_name('td').text)                    

            #check to make sure you're not adding duplicate numbers to the dataframe. 

        if len(personal_phone_numbers) == 0:
            df_phone_numbers.append('None Found')
        elif len(personal_phone_numbers) == 1:
            df_phone_numbers.append(personal_phone_numbers[0])
        else:
            personal_phone_str = '; '.join(str(num) for num in personal_phone_numbers)
            df_phone_numbers.append(personal_phone_str)

        if len(personal_email_addresses) == 0:
            df_email_addresses.append('None Found')
        elif len(personal_email_addresses) == 1:
            df_email_addresses.append(personal_email_addresses[0])
        else:
            personal_email_string = '; '.join(str(email) for email in personal_email_addresses)
            df_email_addresses.append(personal_email_string)

            #print("row: ", str(index), " phone number(s): ", df_phone_numbers[-1], ", email address(es): ", df_email_addresses[-1])

        #t_diff = t_cycle-(datetime.now()-t1)
#        print("going to sleep.")
        time.sleep(random.randint(25,35))
#        print("woke up from sleep at bottom of loop.")

    driver.close()

    eviction_schedule_df['phone_numbers'] = df_phone_numbers
    eviction_schedule_df['email_addresses'] = df_email_addresses

    return eviction_schedule_df

In [ ]:
folks_to_contact = eviction_schedule_df[eviction_schedule_df['phone_numbers'] != 'N/A']

In [ ]:
folks_to_contact.to_csv(r'C:\Users\kevin\Documents\jupyter notebooks\evictions\eviction schedules\\' + begin_date.strftime('%m-%d-%Y eviction schedule.csv'))

# Prepare the text messages to send to renters facing eviction

In [ ]:
#prepare text messages for people.  

#if you need it
#driver = webdriver.Chrome()
filepath = r'C:\Users\kevin\Documents\jupyter notebooks\evictions\eviction schedules\\'

start_date = datetime(year=2020, month=10, day=7)

contact_info_people = pd.read_csv(filepath + start_date.strftime('%m-%d-%Y eviction schedule.csv'))

sender_first_name = 'Kevin'

help_url = 'https://bit.ly/2SkbcZQ'

for index, row in contact_info_people.iterrows():

    if row['phone_numbers'] == 'None Found':
        #contact_info_people.loc[index, "contact_attempt"] = "No cell phone number. Send an email?"
        continue

    phone_str = row['phone_numbers']
    phone_num_list = phone_str.split(';')

    target_first_name = row['first_name'].lower().capitalize()

    text_msg = f"Hi {target_first_name}, I'm {sender_first_name} with DSA Cincinnati/NKY." \
                f" We're watching the county clerk's website for eviction hearings, and I saw your name listed.\n" \
                f"You may be able to halt your eviction per the CDC's moratorium." \
                f" To learn more and see if you qualify click on this link for a pdf from Cincinnati Legal Aid: {help_url}" \
                f"\nNote: I'm not a lawyer, this isn't legal advice, and I'm not affiliated with Cincinnati's Legal Aid Society." \
                f" If you have any non-legal questions, I'll do my best to answer them."   

    for phone_num in phone_num_list:
#        send_text_message(driver, phone_num, text_msg)
        print(f"{phone_num} \n{text_msg}\n\n")
    #contact_info_people.loc[index, "contact_attempt"] = f"Sent messaage to all {len(phone_num_list)} numbers."

